The following code is associated with the manuscript "Missing data imputation via Gaussian Processes: Indo-Aryan sound change," Department of Comparative Linguistics, University of Zurich, 2018.

In [ ]:
import numpy as np
import random
import itertools
from collections import defaultdict
from numpy import log,exp,mean
from functools import reduce
import pymc3 as pm
import theano.tensor as tt
import theano
theano.config.gcc.cxxflags = "-fbracket-depth=1000"
#THEANO_FLAGS = "-fbracket-depth=1000"
from sklearn.metrics import f1_score

Load data and alignment indices:

In [ ]:
text = []

for l in open('cdial_stripped.csv','r'):
    text.append(l.strip().split('\t'))


for i in range(len(text)):
    for j in range(1,3):
        text[i][j] = text[i][j].split()



alignments = []
for l in open('alignments.txt','r'):
    alignments.append([int (i) for i in l.split()])

The following function extracts sound changes from aligned sequences, along with associated variables:

In [ ]:
ng = 2
D = ng*2
def generate_ngrams(ng):
    change_counts = defaultdict(int)
    lang_changes = defaultdict()
    for i in range(len(text)):
        lang = text[i][0]
        if lang not in lang_changes.keys():
            lang_changes[lang] = defaultdict()
        x,y=text[i][2],text[i][1]
        A = alignments[i]
        for j in range(0,len(A)-ng):
            before = []
            after = []
            for k in range(ng):
                before.append(tuple(x[j+k:j+k+1]))
                after.append(tuple(y[A[j+k]:A[j+k+1]]))
            edit = tuple([tuple(before),tuple(after)])
            change_counts[edit]+=1
            if edit[0] not in lang_changes[lang].keys():
                lang_changes[lang][edit[0]] = defaultdict(int)
            lang_changes[lang][edit[0]][edit[1]]+=1

    all_reflex = defaultdict(list)
    for k in change_counts.keys():
        if change_counts[k] > 10:
            all_reflex[k[0]].append(k[1])
            
    reflex = defaultdict(list)
    for k in all_reflex.keys():
        if len(all_reflex[k]) > 1:
            reflex[k] = all_reflex[k]
            
    reflex_list = list(reflex.keys())
    change_list = [(k,v) for k in reflex_list for v in reflex[k]]
    seg_list = [sorted(set([change[i][j] for change in change_list])) for i in range(2) for j in range(ng)]
    seg_len = [len(s) for s in seg_list]
    return(reflex,reflex_list,change_list,lang_changes,seg_list,seg_len)


The following function generates the pairwise segment dissimilarity matrix:

In [ ]:
def generate_dissimilarity():
    feat_mat = {}
    for line in open('feat_matrix.csv','r'):
        l = line.split('\t')
        feat_mat[l[0]]=tuple(l[1:])

    dissimilarity = {}
    dissimilarity[((),())] = 1
    for k in feat_mat.keys():
        dissimilarity[(('#',),(k,))] = 6
        dissimilarity[((k,),('#',))] = 6
        dissimilarity[(('#',),('#',))] = 0
        for l in feat_mat.keys():
            dissim = 0
            if feat_mat[k][0]!=feat_mat[l][0]:
                dissim += 2
            if feat_mat[k][1]!=feat_mat[l][1]:
                dissim += 1
            if feat_mat[k][2]!=feat_mat[l][2]:
                dissim += 1
            if feat_mat[k][3]!=feat_mat[l][3]:
                dissim += 1
            if feat_mat[k][4]!=feat_mat[l][4]:
                dissim += 1
            dissimilarity[((k,),(l,))] = dissim
    return(dissimilarity)

The following function generates the pairwise element dissimilarity matrix for each "dimension" of a given pair of changes.

In [ ]:
def generate_matrix():
    Rho = [np.zeros([len(s),len(s)]) for s in seg_list]
    for i in range(ng):
        for j in range(seg_len[i]):
            for k in range(j+1,seg_len[i]):
                d = dissimilarity[(seg_list[i][j],seg_list[i][k])]
                Rho[i][j,k] = d
                Rho[i][k,j] = d            
            
    for i in range(ng,len(seg_list)):
        for j in range(seg_len[i]):
            for k in range(j+1,seg_len[i]):
                if len(seg_list[i][j]) > 0 and len(seg_list[i][k]) > 0:
                    if len(seg_list[i][j]) > 1 and len(seg_list[i][k]) == 1:
                        d = np.mean([dissimilarity[((r,),seg_list[i][k])] for r in seg_list[i][j]])
                    if len(seg_list[i][j]) == 1 and len(seg_list[i][k]) > 1:
                        d = np.mean([dissimilarity[(seg_list[i][j],(s,))] for s in seg_list[i][k]])
                    if len(seg_list[i][j]) > 1 and len(seg_list[i][k]) > 1:
                        d = np.mean([dissimilarity[((r,),(s,))] for r in seg_list[i][j] for s in seg_list[i][k]])
                    if len(seg_list[i][j]) == 1 and len(seg_list[i][k]) == 1:
                        d = dissimilarity[(seg_list[i][j],seg_list[i][k])]
                else:
                    d = 3
                Rho[i][j,k] = d
                Rho[i][k,j] = d
    return(Rho)

The following function generates a training and held-out data set for a given language, along with associated variables.

In [ ]:
def gen_batch(lang):
    reflex_list_lang = [k for k in reflex_list if k in lang_changes[lang].keys()]
    change_list_lang = [(k,v) for k in reflex_list for v in reflex[k] if k in lang_changes[lang].keys()]
    X = len([k for k in reflex_list_lang])
    R = [len(reflex[k]) for k in reflex_list_lang] #index and length of each sound change distribution
    M = sum([R[i] for i in range(X)])

    C = min(100,int(X/3))
    partition = [[0,R[0]]]+[[reduce(lambda x,y:x+y,R[:i]),reduce(lambda x,y:x+y,R[:i+1])] for i in range(1,len(R))]
    sound_binaries = [np.zeros([M,s]) for s in seg_len]

    for s in range(M):
        for i in range(2):
            for j in range(ng):
                sound_binaries[i*ng+j][s,seg_list[i*ng+j].index(change_list_lang[s][i][j])] = 1
    
    hold_out = sorted(random.sample(range(X),C))
    leave_in = sorted(random.sample([x for x in range(X) if x not in hold_out],C))
    
    #ranges of left in/held out changes
    range_leave_in = [partition[i] for i in leave_in]
    range_hold_out = [partition[i] for i in hold_out]

    #indices
    ind_leave_in = []
    for p in range_leave_in:
        for i in range(p[0],p[1]):
            ind_leave_in.append(i)
        
    ind_hold_out = []
    for p in range_hold_out:
        for i in range(p[0],p[1]):
            ind_hold_out.append(i)        
        
    R_leave_in = [R[i] for i in leave_in]
    R_hold_out = [R[i] for i in hold_out]

    N_leave_in = sum(R_leave_in)
    N_hold_out = sum(R_hold_out)
    
    #absolute partitions, for softmax
    part_leave_in = [[0,R_leave_in[0]]]+[[reduce(lambda x,y:x+y,R_leave_in[:i]),reduce(lambda x,y:x+y,R_leave_in[:i+1])] for i in range(1,len(R_leave_in))]
    part_hold_out = [[0,R_hold_out[0]]]+[[reduce(lambda x,y:x+y,R_hold_out[:i]),reduce(lambda x,y:x+y,R_hold_out[:i+1])] for i in range(1,len(R_hold_out))]
    
    sound_binaries_leave_in = [sound_binaries[i][ind_leave_in,:] for i in range(ng*2)]
    
    change_list_leave_in = [(reflex_list_lang[i],v) for i in leave_in for v in reflex[reflex_list_lang[i]]]
    change_list_hold_out = [(reflex_list_lang[i],v) for i in hold_out for v in reflex[reflex_list_lang[i]]]
    S = len(change_list_leave_in)
    T = len(change_list_hold_out)
    
    sound_count = np.zeros(S)
    for j,c in enumerate(change_list_leave_in):
        sound_count[j] = lang_changes[lang][c[0]][c[1]]
        
    sound_count_held_out = np.zeros(T)
    for j,c in enumerate(change_list_hold_out):
        sound_count_held_out[j] = lang_changes[lang][c[0]][c[1]]
        
    Sigma = np.array([np.dot(np.dot(sound_binaries_leave_in[i],Rho[i]),sound_binaries_leave_in[i].T) for i in range(ng*2)])
    
    sound_binaries_full = [sound_binaries[i][ind_hold_out+ind_leave_in,:] for i in range(ng*2)]
    Sigma_full = np.array([np.dot(np.dot(sound_binaries_full[i],Rho[i]),sound_binaries_full[i].T) for i in range(ng*2)])
    
    return(sound_count,sound_count_held_out,Sigma,Sigma_full,N_leave_in,N_hold_out,part_leave_in,part_hold_out,R_leave_in,R_hold_out)


We generate the data set, the dissimilarity matrix, and the dimension-level dissimilarity matrices:

In [ ]:
reflex,reflex_list,change_list,lang_changes,seg_list,seg_len = generate_ngrams(ng)
dissimilarity = generate_dissimilarity()
Rho = generate_matrix()

The languages we are interesting in are not commented out:

In [ ]:
langs = [#'ashk1246',
 'assa1263',
# 'awad1243',
# 'bagh1251',
# 'balk1252',
 'beng1280',
# 'bhad1241',
# 'bhat1263',
# 'bhoj1244',
# 'braj1242',
# 'brok1247',
# 'carp1235',
# 'cham1307',
# 'chil1275',
# 'chur1258',
# 'dame1241',
# 'dhiv1236',
# 'dogr1250',
# 'doma1258',
# 'doma1260',
# 'garh1243',
# 'gawa1247',
# 'gran1245',
 'guja1252',
# 'halb1244',
 'hind1269',
# 'indu1241',
# 'jaun1243',
# 'kach1277',
# 'kala1372',
# 'kala1373',
# 'kalo1256',
# 'kang1280',
 'kash1277',
# 'kati1270',
# 'khet1238',
# 'khow1242',
# 'kohi1248',
 'konk1267',
# 'kull1236',
# 'kuma1273',
# 'loma1235',
# 'maga1260',
# 'maha1287',
# 'maha1305',
 'mait1250',
# 'malv1243',
# 'mand1409',
 'mara1378',
# 'marw1260',
 'nepa1254',
# 'nort2665',
# 'nort2666',
 'oriy1255',
# 'paha1251',
# 'pali1273',
# 'pang1282',
 'panj1256',
# 'phal1254',
# 'pras1239',
# 'savi1242',
# 'sera1259',
# 'shin1264',
# 'shum1235',
 'sind1272',
 'sinh1246',
# 'sint1235',
# 'sirm1239',
# 'sout2671',
# 'sout2672',
# 'tira1253',
# 'torw1241',
# 'treg1243',
# 'vlax1238',
# 'waig1243',
# 'wels1246',
# 'west2386',
# 'wota1240'
             ]

The following function evaluates the model log-likelihood:

In [ ]:
def logprob(phi):
    def lprob(sounds):
        lp = pm.math.logsumexp(tt.dot(sounds,tt.log(phi.T)))
        return(lp)
    return(lprob)

The following function runs inference for a batch of training data.

In [ ]:
def run_model(N_leave_in,Sigma,part_leave_in,sound_count):
    model = pm.Model()

    with model:
        rho = [pm.Uniform('rho_%i'%d,.001,100000) for d in range(D)]            #length scale        
        alpha = pm.Uniform('alpha',.001,100000)        #dispersion parameter       
        sigma = pm.Uniform('sigma',.001,100000)        #s.d.        
        psi = pm.MvNormal('psi',mu=[0]*N_leave_in,
                       cov = tt.power(alpha,2)*tt.exp(-.5*tt.sum([tt.power(rho[d],-2)*tt.power(Sigma[d],2)
                                               for d in range(D)]))+
                      (np.eye(N_leave_in)*tt.power(sigma,2)),
                     shape=N_leave_in)             #weights       
        phi = tt.concatenate([tt.nnet.softmax(psi[part_leave_in[x][0]:part_leave_in[x][1]])[0] for x in range(len(R_leave_in))])        
        target = pm.DensityDist('target',logprob(phi),observed=sound_count)        
        inference = pm.ADVI()        
        inference.fit(2000,obj_optimizer=pm.adam(learning_rate=.01,beta1=.8),
                                        callbacks=[pm.callbacks.CheckParametersConvergence()])
        trace = inference.approx.sample()
    return(trace)

The following function uses a posterior sample to predict sound changes for held out data:

In [ ]:
def predictive(trace,Sigma_full,N_hold_out,N_leave_in,part_hold_out,R_hold_out,sound_count_held_out):
    j = np.random.randint(500)

    a = trace['psi'][j]
    rho = [trace['rho_'+str(d)][j] for d in range(D)]
    sigma = trace['sigma'][j]
    alpha = trace['alpha'][j]
    
    Sigma_f = (tt.power(alpha,2)*tt.exp(-.5*tt.sum([tt.power(rho[d],-2)*tt.power(Sigma_full[d],2)
                                               for d in range(D)]))+
                      (np.eye(N_leave_in+N_hold_out)*tt.power(sigma,2))).eval()
    
    Sigma_11 = Sigma_f[:N_hold_out,:N_hold_out]
    Sigma_22 = Sigma_f[N_hold_out:,N_hold_out:]
    Sigma_12 = Sigma_f[:N_hold_out,N_hold_out:]
    Sigma_21 = Sigma_f[N_hold_out:,:N_hold_out]
    
    invSigma_22 = np.linalg.inv(Sigma_22)
    CDinv = np.dot(Sigma_12,invSigma_22)
    
    mu_1 = np.matrix(np.zeros(N_hold_out))
    mu_2 = np.matrix(np.zeros(N_leave_in))
    mu_bar = np.array((mu_1.transpose() + (CDinv*(np.matrix(a)-mu_2).transpose())).flatten())[0]
    Sigma_bar = Sigma_11 - np.dot(CDinv,Sigma_21)
    
    psi_22 = np.random.multivariate_normal(mu_bar,Sigma_bar)
    phi_22 = tt.concatenate([tt.nnet.softmax(psi_22[part_hold_out[x][0]:part_hold_out[x][1]])[0] for x in range(len(R_hold_out))]).eval()
    
    N = [sum(sound_count_held_out[part_hold_out[x][0]:part_hold_out[x][1]]) for x in range(len(R_hold_out))]
    
    Z = tt.concatenate([np.random.multinomial(N[x],phi_22[part_hold_out[x][0]:part_hold_out[x][1]]) for x in range(len(R_hold_out))]).eval()
    
    F_score = f1_score(Z,sound_count_held_out,average='micro')
    f = open('f_score.txt','a')
    print(lang,F_score,file=f)
    f.close()


The full inference and evaluation procedure:

In [ ]:
for t in range(100):
    for lang in langs:
        sound_count,sound_count_held_out,Sigma,Sigma_full,N_leave_in,N_hold_out,part_leave_in,part_hold_out,R_leave_in,R_hold_out = gen_batch(lang)
        trace=run_model(N_leave_in,Sigma,part_leave_in,sound_count)
        predictive(trace,Sigma_full,N_hold_out,N_leave_in,part_hold_out,R_hold_out,sound_count_held_out)
